In [4]:
#!pip install textblob

In [1]:
from textblob import TextBlob
#from attributegetter import *
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random
import os
import re

In [2]:
def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''

    context = IntentComplete()
    return 'action: ' + current_intent.action, context

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    #print("intent in check required")
    #print(str(current_intent))
    #print(str(attributes))
    
    for para in current_intent.params:
        #print("para")
        #print(para)
        if para.required:
            if para.name not in attributes:
                #Example of where the context is born, implemented in Contexts.py
                if para.name=='RegNo':
                    context = GetRegNo()
                elif para.name=='Book_Title':
                    context = Book_Title()
                elif para.name=='Book_Author':
                    context = Book_Author()
                elif para.name=='Book_RestType':
                    context = Book_RestType()
                elif para.name=='Book_RestLoc':
                    context = Book_RestLoc()
                elif para.name=='Book_RestRange':
                    context = Book_RestRange()
                elif para.name=='Clarity':
                    context = Clarity()
                #else:
                    #print("should not come here")
                    #restart()
                #returning a random prompt frmo available choices.
                return random.choice(para.prompts), context

    return None, context


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
#     attributes, cleaned_input = getattributes(user_input, context, attributes)
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    
    
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    #print( "clean input is " + str(clean_input))
    
    scores,h_class = ngrammatch(clean_input) #Seema changed the algorithm
    #scores = classify(clean_input) #Seema added
    
    #choosing here the intent with the highest score
    #scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    #print("highest intent is")
    #print(h_class)
    #print("highest score")
    #print(scores)

    if(current_intent==None):
        if h_class=="RestSearch":
            #print("in search")
            return loadIntent('params/newparams.cfg', 'RestSearch')
        elif h_class=="OrderBook":
            #print("in book")
            return loadIntent('params/newparams.cfg','OrderBook')
        else:
            return loadIntent('params/newparams.cfg','Clarity')
            #return loadIntent('params/newparams.cfg', 'RestSearch')
    else:
        #If current intent is not none, stick with the ongoing intent
        return current_intent

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    
    #Can use context to context specific attribute fetching
    if (uinput is None):
        restart()
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    
    else:
        #Code can be optimised here, loading the same files each time suboptimal 
        files = os.listdir('./entities/')
        entities = {}
        for fil in files:
#             entities[str(fil)[:-4]] = []
            lines = open('./entities/'+fil).readlines()
            for i, line in enumerate(lines):
                lines[i] = line[:-1]
            entities[fil[:-4]] = '|'.join(lines)
            
        
        #Extract entity and update it in attributes dict
        for entity in entities:
            #print(entity)
            for i in entities[entity].split('|'):
                if i.lower() in uinput.lower():
                    attributes[entity] = i
        for entity in entities:
                uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)

        
        #Example of where the context is being used to do conditional branching.
        #if context.name=='GetRegNo' and context.active:
        
        match = re.search('[0-9]+', uinput)
        #print(str(match))
        if match:
#             uinput = re.sub('[0-9]+', '$regno', uinput)
            attributes['RegNo'] = match.group()
            context.active = False
    #elif context.name=='Book_Title' and context.active:
        
        match = re.search('[a-zA-Z0-9_]+', uinput)
        #print("Type of match.group 1  " + str(type(match.group())))
        if match:
            #print("Type of match.group 2 " + str(type(match.group())))
            titles = entities["title"]
            for title in titles:
                if title == match.group():
                    attributes['title'] = match.group()
                    context.active = False
#             uinput = re.sub('[0-9]+', '$title', uinput)
            
    #elif context.name=='Book_Author' and context.active:
        
        match = re.search('[a-zA-Z]+', uinput)
        if match:
            authors = entities["author"]
            for author in authors:
                if author == match.group():
                    attributes['author'] = match.group()
#                     context.active = False
    #elif context.name=='Book_RestType' and context.active:
        
        match = re.search('[a-zA-Z]+', uinput)
        if match:
            types = entities["type"]
            for typ in types:
                if typ == match.group():
                    attributes['type'] = match.group()
                    context.active = False
    #elif context.name=='Book_RestLoc' and context.active:
        match = re.search('[a-zA-Z]+', uinput)
        if match:
            locations = entities["location"]
            for loc in locations:
                if loc == match.group():
                    attributes['location'] = match.group()
                    context.active = False                    
        
        match = re.search('[a-zA-Z]+', uinput)
        if match:
            ranges = entities["range"]
            for rang in ranges:
                if rang == match.group():
                    attributes['range'] = match.group()
                    context.active = False                    
        
    #else:
        #print("Why did you come here")
        #print(context.name)
        #print(context.active)
        #print(attributes)
        return attributes, uinput

In [3]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
        
        #Contexts are flags which control dialogue flow, see Contexts.py        
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def getIntent(self, user_input):
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        return self.attributes, self.current_intent
    
    
    def reply(self):
        
        '''Generate response to user input'''
        
#         self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
#         self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [ ]:
session = Session()


def restart():
    try:
        inp = input('User: ')
        if not inp:
            raise ValueError('Empty String')
            print ('BOT: Hi! How may I assist you?')
        session.getIntent(inp)
#         print ('BOT:', session.reply(inp))
        print ('BOT:', session.reply())
    except ValueError as e:
        print(e)
    
    
    #print ('-----BOT: Hi! How may I assist you?')

    
#session = Session(FirstGreeting())




if __name__ == '__main__':
    print ('BOT: Hi! How may I assist you?')



    while True:
        try:
            inp = input('User: ')
            #restart()
            if not inp:
                raise ValueError('Empty String')
            session.getIntent(inp)
#         print ('BOT:', session.reply(inp))
            print ('BOT:', session.reply())
    
        except ValueError as e:
            print(e)
        
    
    
    
    
            

#     inp = input('User: ')
#     print ('BOT:', session.reply(inp))

BOT: Hi! How may I assist you?
User: book
BOT: What's the name of the book?
User: harry potter
BOT: Who wrote this book?
User: indira gandhi
Hi
BOT: Please type your registration number
User: 223344
BOT: action: BookIssue
User: food
BOT: What type of cusine you want?
User: indian
BOT: Do you want in East or West or North or South?
User: west
BOT: Do you need a cheap, medium or expensive restaurant?
User: cheap
BOT: What kind of restaurant you want?
User: low
BOT: action: SearchRest
User: dine
BOT: Do you prefer chinese or Indian or Mexican or Italian etc.?
User: chinese
BOT: Do you want in East or West or North or South?
User: west
BOT: Do you need a cheap, medium or expensive restaurant?
User: low
BOT: action: SearchRest
User: buy book
BOT: What's the name of the book?
User: 
Empty String
User: alksjdjakl
BOT: What's the name of the book?
